In [7]:
import numpy as np
import pandas as pd
import json

from create_lens_pop import CreateLensPop
from astropy.cosmology import FlatLambdaCDM
from datetime import date
import sys

In [6]:
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=0.05)

Example with built in functions for elliptical NFW halo and galaxy modeled as two sersic profiles for disk and bulge separately.

create_lens_pop will search for built in functions are already in create_lens_model.py that are associated with "nfw_ellipse", "sersic_ellipse_disk", and "sersic_ellipse_sphere".

As will be shown, each galaxy or halo can have multiple components, just specify more halo types and functions in input list.

In [ ]:
# number of lenses to simulate
n_lens = 100

# read parameters from json
f = open('lens_model_input.json')
jf = json.load(f)

# minumum and maximum redshift range for lenses
z_min = jf['z_min']
z_max = jf['z_max']

gal_halo_mass_min = 10**jf['gal_halo_mass_min']
gal_halo_mass_max = 10**jf['gal_halo_mass_max']

halo_type = jf['halo_type']  # ["NFW_ELLIPSE"]
halo_function = jf['halo_function']  # ["nfw_ellipse"]
halo_catalog_name = jf['halo_catalog']  # mini_halo_cat.csv
galaxy_type = jf['galaxy_type']  # ["SERSIC_ELLIPSE_POTENTIAL", "SERSIC_ELLIPSE_POTENTIAL"],
galaxy_function = jf['galaxy_function']  # ["sersic_ellipse_disk","sersic_ellipse_sphere"],
galaxy_catalog_name = jf['galaxy_catalog']  # mini_gal_cat.csv
source_catalog_name = jf['source_catalog']  # mini_source_cat.csv
shear = jf['shear']  # true

galaxy_catalog = pd.read_parquet(galaxy_catalog_name)
source_catalog = pd.read_pickle(source_catalog_name)
halo_catalog = pd.read_parquet(halo_catalog_name)

In [2]:
def main(clp):

    df = clp.monte_carlo(z_lens_min=z_min, z_lens_max=z_max)

    return df

In [ ]:
clp = CreateLensPop(cosmo=cosmo,
                    halo_catalog=halo_catalog,
                    galaxy_catalog=galaxy_catalog,
                    source_catalog=source_catalog,
                    halo_type=halo_type,
                    galaxy_type=galaxy_type,
                    galaxy_function=galaxy_function,
                    halo_function=halo_function,
                    shear=True)

In [ ]:
# define empty dataframe for collecting output
df = pd.DataFrame()

# loop through number of lenses
for n in range(n_lens):
    res = main(clp)
    # add on to dataframe
    df = pd.concat((df, res))

Example implementing custom functions.

Only requirement is that this function must be compatible with lenstronomy mass models.

In [ ]:
#Example custom function titled custom_function.py
import lenstronomy.Util.param_util as param_util

def nfw_ellipse_create_kwargs(LC, properties, x, y):
    """
        Create all necessary arguments needed as input for lenstronomy halos
        Need Rs and alpha Rs in arcsecs!

        Args:
            LC (object): lens cosmo (from lenstronomy)
            properties (pandas dataframe): dataframe with lens halo properties

        Returns:
            list: list containing dictionary of args to be used as input for lenstronomy
        """

    # get keys
    keys = properties.keys()

    if np.logical_and('Rs_angle' in keys, 'alpha_Rs' in keys):
        Rs_angle = properties['Rs_angle'].values[0]
        alpha_Rs = properties['alpha_Rs'].values[0]

    else:
        # calculate scale radius and deflection angle at scale radius in arcsecs
        c = properties['c'].values[0]
        m200 = properties['m200'].values[0]

        Rs_angle, alpha_Rs = LC.nfw_physical2angle(M=m200, c=c)

    # ellipticity parameters used by lenstronomy for elliptical NFW
    if np.logical_and('e1' in keys, 'e2' in keys):
        e1 = properties['e1'].values[0]
        e2 = properties['e2'].values[0]

    # if e1 and e2 are not known, calculate using position angle and axis ratio
    else:
        # axis ratio
        q = properties['q'].values[0]
        # major axis direction from E to N, set 0 for just rotating about one axis (z-axis)
        # phi is in radians
        phi = properties['position_angle'].values[0]
        e1, e2 = param_util.phi_q2_ellipticity(phi=phi, q=q)

        # create kwargs
    halo_kwargs = {
        "Rs": Rs_angle,
        "alpha_Rs": alpha_Rs,
        "e1": e1,
        "e2": e2,
        "center_x": x,
        "center_y": y
    }

    return halo_kwargs

In this case, user would input halo_type as "CUSTOM_FUNCTION" and then supply the name of the python script with the custom function, for example custom_function.py, as custom_halo_function.


In [ ]:
# number of lenses to simulate
n_lens = 100

# read parameters from json
f = open('lens_model_input_custom_function.json')
jf = json.load(f)

# minumum and maximum redshift range for lenses
z_min = jf['z_min']
z_max = jf['z_max']

gal_halo_mass_min = 10**jf['gal_halo_mass_min']
gal_halo_mass_max = 10**jf['gal_halo_mass_max']


halo_catalog_name = jf['halo_catalog']  # mini_halo_cat.csv
galaxy_catalog_name = jf['galaxy_catalog']  # mini_gal_cat.csv
source_catalog_name = jf['source_catalog']  # mini_source_cat.csv


halo_type = jf['halo_type']  # ["NFW_ELLIPSE"]
halo_function = jf['halo_function']  # ["CUSTOM_FUNCTION"]
custom_halo_function = jf['halo_custom_function'] #['custom_function.py']
galaxy_type = jf['galaxy_type']#["SERSIC_ELLIPSE_POTENTIAL", "SERSIC_ELLIPSE_POTENTIAL"]
galaxy_function = jf['galaxy_function']#["sersic_ellipse_disk", "sersic_ellipse_sphere"]

shear = jf['shear']  # true

In [ ]:
clp = CreateLensPop(cosmo=cosmo,
                    halo_catalog=halo_catalog,
                    galaxy_catalog=galaxy_catalog,
                    source_catalog=source_catalog,
                    halo_type=halo_type,
                    halo_function=halo_function,
                    custom_halo_function=custom_halo_function,
                    galaxy_type=galaxy_type,
                    galaxy_function=galaxy_function,
                    halo_function=halo_function,
                    shear=True)